In [ ]:
complete_date_range = pd.date_range(start=df_weekly.index.min(), end=df_weekly.index.max(), freq='D')

In [ ]:
complete_df = pd.DataFrame(index=complete_date_range)
# Merge the complete DataFrame with the original DataFrame
merged_df = pd.merge(df_weekly, complete_df, left_index=True, right_index=True, how='right')
merged_df['Quantity'] = merged_df['Quantity'].fillna(0)
merged_df = merged_df.resample('W').sum()

In [ ]:
df_reset = merged_df.reset_index()
df_reset.rename(columns={'index': 'Date'}, inplace=True)
df_reset

In [ ]:
import numpy as np
import pandas as pd
#from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
import warnings
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

In [ ]:
scaler = Scaler()
series = scaler.fit_transform(

       TimeSeries.from_dataframe(df_reset, 'Date', 'Quantity', freq='w')
    
).astype(np.float32)
series.plot()

In [ ]:
training_cutoff = int(len(series)*0.2)
train,val = series[:-training_cutoff],series[-training_cutoff:]

In [ ]:
input_chunk_length = 5
forecast_horizon = 1
model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=forecast_horizon,
    hidden_size=150,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=16,
    # n_epochs=80,
    nr_epochs_val_period = 10,
    add_relative_index=True,
    add_encoders=None,
    random_state=42,
)

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor='val_loss',    
    patience=10,            
    min_delta=0.001,        
    mode='min'              
)

In [ ]:
trainer_1 = Trainer(
    max_epochs=200,         # Number of epochs
    callbacks=[early_stopping_callback]  # Pass the EarlyStopping callback
)

In [ ]:
model.fit(train,val_series=val,trainer=trainer_1) #, future_covariates=covariates_transformed, verbose = True)

In [ ]:
extra_data=model.predict(len(train))

In [ ]:
output=model.predict(len(train)-5,train)

In [ ]:
original_series = scaler.inverse_transform(train)
original_series_output = scaler.inverse_transform(output)
original_series.plot(label="Actual")
original_series_output.plot(label = "Forecast")

In [ ]:
mse = mean_squared_error(pd.DataFrame(original_series.values())[5:], original_series_output.values())
print("Mean Squared Error:", mse)